In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import glob
import cartopy.crs as ccrs
import cmocean

In [ ]:
import sys
sys.path.append("scripts/")
import processing as proc
import spectrumfct as spectrum

In [ ]:
from nitime.algorithms.spectral import multi_taper_csd

In [ ]:
rho0 = 1026 # Reference density [kg/m^3]

In [ ]:
ref_lat = 40 # Reference latitude for AMOC calculations (°N)

### Import AMOC fields

In [ ]:
model_list = pd.read_csv("model_list.csv").set_index("Model")
sel_models = list(model_list.index)

In [ ]:
# Import streamfunction fields
amoc_strf = {}
amoc_strf_detr = {}
for model in sel_models:
    member = model_list.loc[model,"Member"]
    amoc_files = glob.glob("/home/omehling/work/cmip6/piControl_processed/amoc/CMIP6_{}_*{}*.nc".format(model, member))
    if len(amoc_files) != 1:
        raise ValueError("{} files found for {}".format(len(amoc_files), model))
    ds_imp = xr.open_dataset(amoc_files[0], use_cftime=True)
    # Rename "rlat" to "lat" if needed
    if 'rlat' in list(ds_imp.dims):
        ds_imp = ds_imp.rename({'rlat':'lat'})
    # Select streamfunction variable
    moc_var_name = "msftmz" if "msftmz" in list(ds_imp.data_vars) else "msftyz"
    amoc_strf[model] = ds_imp[moc_var_name].drop("sector")/rho0*1e-6 # Units: kg/s -> Sv
    amoc_strf_detr[model] = proc.detrend_xr(amoc_strf[model], 2) # Quadratic detrending
    
    print(model)

### AMOC time series

(Suppl. Fig. S4)

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(15,5),sharex=True)
for model_id, ax in zip(sel_models, axes.flat):
    amoc_index = amoc_strf_detr[model_id].sel(lat=ref_lat, method="nearest").sel(lev=slice(500,None)).max("lev")
    amoc_index_lf = proc.filter_xr(amoc_index, 70, detrend=False)
    amoc_mean = amoc_index.mean("time").item()
    amoc_std_lf = amoc_index_lf.std("time").item()
    #print('{}: std={:.2f}'.format(model_id, amoc_std_lf))

    ax.plot(amoc_index, lw=.3, c='C7')
    ax.plot(amoc_index_lf, lw=1, c='k')
    ax.text(.98,.1,r'$\sigma_{LF}=$'+'{:.2f} Sv'.format(amoc_std_lf),transform=ax.transAxes,ha='right')
    ax.set(xlim=(0,2000),xlabel='',ylabel='',ylim=[amoc_mean-5.5, amoc_mean+5.5])
    ax.plot(amoc_index_lf.where(amoc_index_lf<(amoc_mean-amoc_std_lf)), c='C0')
    ax.plot(amoc_index_lf.where(amoc_index_lf>(amoc_mean+amoc_std_lf)), c='C3')
    ax.set_title(model_id,y=.9)
    ax.set_yticks(np.arange(int(((amoc_mean-5.5)//2+1)*2), int(((amoc_mean+5.5)//2+1)*2), 2))
    ax.set_xticks(np.arange(0,2100,500))
    sns.despine()
fig.supylabel('AMOC strength at 26.5°N [Sv]',x=.09,size=10)
fig.supxlabel('Time [model years]',size=10,y=0.0);
#fig.suptitle('CMIP6 piControl AMOC timeseries at low-frequency maximum',y=.96)

fig.savefig('figures/Suppl_AMOC-timeseries.pdf',bbox_inches='tight')

In [ ]:
fig, axes = plt.subplots(9, 1, figsize=(8,12),sharex=True)
for model_id, ax in zip(sel_models, axes.flat):
    amoc_index = amoc_strf_detr[model_id].sel(lat=ref_lat, method="nearest").sel(lev=slice(500,None)).max("lev")
    amoc_index_lf = proc.filter_xr(amoc_index, 70, detrend=False)
    amoc_mean = amoc_index.mean("time").item()
    amoc_std_lf = amoc_index_lf.std("time").item()
    #print('{}: std={:.2f}'.format(model_id, amoc_std_lf))

    ax.plot(amoc_index, lw=.3, c='C7')
    ax.plot(amoc_index_lf, lw=1.4, c='k')
    ax.text(.98,.1,r'$\sigma_{LF}=$'+'{:.2f} Sv'.format(amoc_std_lf),transform=ax.transAxes,ha='right')
    ax.set(xlim=(0,2000),xlabel='',ylabel='',ylim=[amoc_mean-5.5, amoc_mean+5.5])
    ax.plot(amoc_index_lf.where(amoc_index_lf<(amoc_mean-amoc_std_lf)), c='C0', lw=1.4)
    ax.plot(amoc_index_lf.where(amoc_index_lf>(amoc_mean+amoc_std_lf)), c='C3', lw=1.4)
    ax.set_title(model_id,y=.86,size=11)
    ax.set_yticks(np.arange(int(((amoc_mean-5.5)//2+1)*2), int(((amoc_mean+5.5)//2+1)*2), 2))
    ax.set_xticks(np.arange(0,2100,500))
    sns.despine()
fig.supylabel('AMOC strength at 26.5°N [Sv]',x=.06,size=10)
fig.supxlabel('Time [model years]',size=10,y=0.07);
#fig.suptitle('CMIP6 piControl AMOC timeseries at low-frequency maximum',y=.96)

fig.savefig('figures/Suppl_AMOC-timeseries-stacked.pdf',bbox_inches='tight')

### AMOC power spectra for each latitude

(Suppl. Fig. S2)

In [ ]:
def xrspec(ts, sampling_interval=1):
    spec=spectrum.MTMSpectrum(ts, sampling_interval=sampling_interval)
    spec.estimate(NW=3)
    return xr.DataArray(
        data=spec.psd,
        coords={'freq':spec.freqs}
    )

def xrspec_conf(ts, ci=.99, dfsmooth=.05, sampling_interval=1):
    spec=spectrum.MTMSpectrum(ts, sampling_interval=sampling_interval)
    spec.estimate(NW=3)
    spec.fit(dfsmooth=dfsmooth)
    lower_ci, upper_ci = spec.getCIs(ci=ci)
    return xr.DataArray(
        data=upper_ci,
        coords={'freq':spec.freqs}
    )

In [ ]:
latspecs={}
latspecs_conf={}

for i, model_id in enumerate(sel_models):
    ds_latamoc = amoc_strf_detr[model_id].sel(lat=slice(-34,80)).sel(lev=slice(500,None)).max("lev").dropna('lat')

    # Spectrum by lat
    ds_latspec=xr.apply_ufunc(
        xrspec,
        ds_latamoc,
        input_core_dims=[["time"]],
        exclude_dims=set(("time",)),
        output_core_dims=[["freq"]],
        vectorize=True
    )
    ds_latspec['freq']=xrspec(ds_latamoc.isel(lat=0).values).freq
    # Upper CI by lat
    ds_latspec_conf=xr.apply_ufunc(
        xrspec_conf,
        ds_latamoc,
        input_core_dims=[["time"]],
        exclude_dims=set(("time",)),
        output_core_dims=[["freq"]],
        vectorize=True
    )
    ds_latspec_conf['freq']=xrspec(ds_latamoc.isel(lat=0).values).freq
    # Add to list
    latspecs[model_id]=ds_latspec
    latspecs_conf[model_id]=ds_latspec_conf

    print(model_id)

In [ ]:
with plt.rc_context({"hatch.color": "#777777", "hatch.linewidth": 0.3}):
    fig, axes = plt.subplots(3, 3, figsize=(9, 6),sharex=True,sharey=True)
    for model_id, ax in zip(sel_models, axes.flat):
        latspec_sel = latspecs[model_id].isel(freq=slice(1,None))
        c = ax.pcolormesh(1/latspec_sel.freq.values, latspec_sel.lat, latspec_sel.values, cmap='cmo.dense', vmin=0)
        #c = latspecs[model_id].isel(freq=slice(1,None)).plot(x='freq', cmap='cmo.dense', vmin=0, ax=ax, add_colorbar=False)
        signif_contours = (latspecs[model_id].where(latspecs[model_id]>latspecs_conf[model_id])*0+1).fillna(0).isel(freq=slice(1,None))
        ax.contourf(1/latspec_sel.freq.values, latspec_sel.lat, signif_contours.values,
                    levels=[-1,0,1], hatches=[None,"....",None], alpha=0, colors=None)    
        #signif_contours.plot.contour(x='freq',levels=[.5],colors='k',linewidths=.5,ax=ax)
        #signif_contours.plot.contourf(levels=[-1,0,1], hatches=[None,"....",None], alpha=0, colors=None, add_colorbar=False, ax=ax)    
        #latspecs[model_id].isel(freq=slice(1,None)).plot(x='freq', cmap='cmo.dense', alpha=0.6, vmin=0, ax=ax, add_colorbar=False)
        plt.colorbar(c)

        ax.axhline(40, c="k", lw=.7, ls="--")
        ax.set(xlabel="", ylabel="", yticks=np.arange(-20,70,20), xscale='log',xlim=(1000,10))#xlim=(1/1000,1/10))
        ax.set_title(model_id, size=10)

    fig.supylabel('Latitude [°N]',x=0.05,size=10)
    fig.supxlabel('Frequency [model years]',size=10,y=0.02);
    fig.savefig("figures/Suppl_Spectra_latitude.png", bbox_inches="tight", dpi=400)

### AMOC spectra at 40°N (Fig. 1)

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(10,7), sharex=True, sharey=True)
cpal=sns.color_palette("husl", len(sel_models))
for i, model_id in enumerate(sel_models):
    ax=axes.flat[i]

    amoc_index = amoc_strf_detr[model_id].sel(lat=ref_lat, method="nearest").sel(lev=slice(500,None)).max("lev")
    amoc_spec = spectrum.MTMSpectrum(amoc_index.values, sampling_interval=1.)
    amoc_spec.estimate(NW=3)
    amoc_spec.plot_spectrum(ax=ax, x='period', lw=.3, c='#aaaaaa')
    amoc_spec.fit(dfsmooth=.05)
    _, upper_ci = amoc_spec.getCIs(.99)
    amoc_spec.plot_CIs(.99, 'upper', c='k', ls=':', x='period', ax=ax, lw=1)
    masked_psd = np.ma.masked_less(amoc_spec.psd, upper_ci)
    ax.plot(1/amoc_spec.freqs[1:], masked_psd[1:], c=cpal[i], lw=2)

    ax.axvspan(250, 100, color='#f2f2f2')
    ax.set(xlim=(2000,2), xlabel='', ylabel='')
    ax.set_title(model_id,y=.98)
fig.subplots_adjust(hspace=.15, wspace=.1)
fig.supxlabel('Period [yr]', fontsize=10, y=.04)
fig.supylabel('Spectral power [Sv$^2$]', fontsize=10, x=.05)
sns.despine()
fig.savefig('figures/amoc-spectra-1000ymodels.pdf', bbox_inches='tight')

### AMOC coherence by latitude

(Suppl. Fig. S3)

In [ ]:
csd_amoc = {}
for i, model_id in enumerate(sel_models):
    ds_latamoc = amoc_strf_detr[model_id].sel(lat=slice(-34,80)).sel(lev=slice(500,None)).max("lev").dropna('lat')
    cfreq, csd = multi_taper_csd(np.asarray([
            ds_latamoc.sel(lat=ref_lat,method='nearest').values,
            ds_latamoc.sel(lat=26.5,method='nearest')
        ]), Fs=1., NW=3) # one test iteration
    
    csd_amoc_sel = np.zeros((len(ds_latamoc.lat),2,2,len(cfreq)), dtype=np.complex64)
    for i, sel_lat in enumerate(ds_latamoc.lat.values):
        cfreq, csd = multi_taper_csd(np.asarray([
            ds_latamoc.sel(lat=ref_lat,method='nearest').values,
            ds_latamoc.sel(lat=sel_lat,method='nearest')
        ]), Fs=1., NW=3)
        csd_amoc_sel[i,:,:,:] = csd[:,:,:]
    freq_sel = np.where(np.logical_and(cfreq>1/150, cfreq<1/100))
    csd_amoc_sel = np.squeeze(np.mean(csd_amoc_sel[:,:,:,freq_sel], axis=4))
    csd_amoc[model_id] = csd_amoc_sel
    
    print(model_id)

In [ ]:
fig, axes = plt.subplots(2,1,figsize=(6,7), sharex=True)
cpal = sns.color_palette("husl", len(sel_models))
for i, model_id in enumerate(sel_models):
    lat_sel = amoc_strf_detr[model_id]["lat"].sel(lat=slice(-34,80)).dropna('lat')
    coh_amoc = np.abs(csd_amoc[model_id][:,0,1]**2) / np.real(csd_amoc[model_id][:,0,0]*csd_amoc[model_id][:,1,1])
    coh_amoc = xr.DataArray(coh_amoc, coords={"lat": lat_sel})
    coh_amoc.sel(lat=slice((-30 if model_id=="CanESM5" else -34),55)).plot(label=model_id, ax=axes[0], c=cpal[i])
    angle_amoc = np.angle(csd_amoc[model_id][:,0,1])/np.pi*180
    angle_amoc = xr.DataArray(angle_amoc, coords={"lat": lat_sel})
    angle_amoc.sel(lat=slice((-30 if model_id=="CanESM5" else -34),55)).plot(ax=axes[1], c=cpal[i])
#axes[0].legend()
axes[0].axhline(1, c="k", lw=.7)
axes[0].axvline(40, c="k", lw=.7)
axes[0].set(ylim=(0,1.05), xlim=(-34,55), xlabel="", ylabel="Coherence with AMOC at 40°N")
axes[0].set_title("AMOC strength: Coherence")
axes[1].axhline(0, c="k", lw=.7)
axes[1].axvline(40, c="k", lw=.7)
axes[1].set(ylim=(-90,90), yticks=np.arange(-90,100,30), xlim=(-34,55), xlabel="Latitude [°N]", ylabel="Angle w.r.t. AMOC at 40°N")
axes[1].set_title("AMOC strength: Angle")
fig.legend(bbox_to_anchor=(0.92,0.5), loc="center left", frameon=False)
sns.despine()

fig.savefig("figures/Suppl_AMOC_coherence.pdf", bbox_inches="tight")